<a href="https://colab.research.google.com/github/polrgn/Classification-tree-model/blob/master/Paralllel_Boosting_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#if(!require("knitr")) install.packages("knitr")
if(!require("tree")) install.packages("tree")
if(!require("readr")) install.packages("readr")
#if(!require("knitr")) install.packages("knitr")
if(!require("caret")) install.packages("caret")
#if(!require("gridExtra")) install.packages("gridExtra")
if(!require("randomForest")) install.packages("randomForest")
if(!require("foreach")) install.packages("foreach")
if(!require("e1071")) install.packages("e1071")
if(!require("furrr")) install.packages("furrr")
if(!require("parallel")) install.packages("parellel")
if(!require("doParallel")) install.packages("doParallel")
if(!require("gbm")) install.packages("gbm")




Loading required package: randomForest

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:dplyr’:

    combine


The following object is masked from ‘package:ggplot2’:

    margin


Loading required package: e1071

Loading required package: gbm

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘gbm’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘gridExtra’




In [0]:
require("readr")
require("tidyverse")
#require("knitr")
require("caret")
require("parallel")
require("doParallel")


In [6]:
# data parsing
soldat <- read_csv("/soldat.csv")
soldat$y <- factor(soldat$y,levels = c(-1,1),labels = c("insoluble","soluble"))
n <- nrow(soldat)
p <- ncol(soldat)

Parsed with column specification:
cols(
  .default = col_double()
)

See spec(...) for full column specifications.



In [0]:
cluster <- makeCluster(detectCores() - 1) # convention to leave 1 core for OS
registerDoParallel(cluster)

In [0]:
v <- cor(soldat %>% select(-y), use = "pairwise.complete.obs")
high_corr <- colnames(soldat)[findCorrelation(v, cutoff = 0.95)]
soldat <- soldat %>% select(-all_of(high_corr))

In [0]:
set.seed(1234)
inTest <- createDataPartition(soldat$y,p = 0.5, list = FALSE)[,1]
test <- soldat[inTest,]
training <- soldat[-inTest,]

In [0]:
control <- trainControl(method = "cv", 
                       number = 5,
                       classProbs = TRUE, 
                       allowParallel = TRUE)
metric <- "Accuracy"

grid_ada <- expand.grid(.interaction.depth = 1, .n.trees = c(1, 10, 100, 250, 500, 750, 1000, 1250, 1500, 1750, 2000),
 .shrinkage = c(.0001,.001,.01,.1,1), .n.minobsinnode = 10)

adaboost_stump <- train(y ~. , data = training, 
                             method = "gbm", 
                             trControl = control,
                             tuneGrid = grid_ada,
                             distribution = "adaboost",
                             metric = metric,
                             verbose = FALSE)
save(adaboost_stump,file="adaboost_stump.Rdata")

In [34]:
# varying the interaction depth
grid_ada <- expand.grid(.interaction.depth = c(1,4,8,16), .n.trees = c(10,100,1000,2000),
 .shrinkage = c(.0001,.001,.01,.1,1), .n.minobsinnode = 10)
adaboost_models <- train(y ~. , data = training, 
                             method = "gbm", 
                             trControl = control,
                             tuneGrid = grid_ada,
                             distribution = "adaboost",
                             metric = metric,
                             verbose = FALSE)
save(adaboost_models,file="bt_models.Rdata")

Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1        0.9706            -nan     0.0001    0.0000
     2        0.9706            -nan     0.0001    0.0000
     3        0.9706            -nan     0.0001    0.0000
     4        0.9706            -nan     0.0001    0.0000
     5        0.9706            -nan     0.0001    0.0000
     6        0.9705            -nan     0.0001    0.0000
     7        0.9705            -nan     0.0001    0.0000
     8        0.9705            -nan     0.0001    0.0000
     9        0.9705            -nan     0.0001    0.0000
    10        0.9705            -nan     0.0001    0.0000
    20        0.9704            -nan     0.0001    0.0000
    40        0.9702            -nan     0.0001    0.0000
    60        0.9700            -nan     0.0001    0.0000
    80        0.9698            -nan     0.0001    0.0000
   100        0.9696            -nan     0.0001    0.0000
   120        0.9693            -nan     0.0001    0.0000
   140        

In [37]:
print(adaboost_stump)

Stochastic Gradient Boosting 

2815 samples
  46 predictor
   2 classes: 'insoluble', 'soluble' 

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 2252, 2252, 2251, 2253, 2252 
Resampling results across tuning parameters:

  shrinkage  n.trees  Accuracy   Kappa    
  1e-04         1     0.6202488  0.0000000
  1e-04        10     0.6202488  0.0000000
  1e-04       100     0.6202488  0.0000000
  1e-04       250     0.6202488  0.0000000
  1e-04       500     0.6202488  0.0000000
  1e-04       750     0.6202488  0.0000000
  1e-04      1000     0.6202488  0.0000000
  1e-04      1250     0.6202488  0.0000000
  1e-04      1500     0.6202488  0.0000000
  1e-04      1750     0.6202488  0.0000000
  1e-04      2000     0.6202488  0.0000000
  1e-03         1     0.6202488  0.0000000
  1e-03        10     0.6202488  0.0000000
  1e-03       100     0.6202488  0.0000000
  1e-03       250     0.6202488  0.0000000
  1e-03       500     0.6202488  0.0000000
  1e-03       

In [35]:
print(adaboost_models)

Stochastic Gradient Boosting 

2815 samples
  46 predictor
   2 classes: 'insoluble', 'soluble' 

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 2252, 2252, 2251, 2253, 2252 
Resampling results across tuning parameters:

  shrinkage  interaction.depth  n.trees  Accuracy   Kappa    
  1e-04       1                   10     0.6202488  0.0000000
  1e-04       1                  100     0.6202488  0.0000000
  1e-04       1                 1000     0.6202488  0.0000000
  1e-04       1                 2000     0.6202488  0.0000000
  1e-04       4                   10     0.6202488  0.0000000
  1e-04       4                  100     0.6202488  0.0000000
  1e-04       4                 1000     0.6202488  0.0000000
  1e-04       4                 2000     0.6202488  0.0000000
  1e-04       8                   10     0.6202488  0.0000000
  1e-04       8                  100     0.6202488  0.0000000
  1e-04       8                 1000     0.6202488  0.0000000
 